In [11]:
import math
from intervaltree import Interval, IntervalTree
from collections import defaultdict

import polars as pl
import matplotlib.pyplot as plt
from scipy import signal

plt.rcParams["figure.figsize"] = [16, 5]

In [12]:
df = pl.read_csv(
    "../../test/cdr/input/HG00731_intersect.bed",
    separator="\t",
    has_header=False,
    new_columns=["chrom", "st", "end", "avg", "idx"],
)
median_filt_window: int | None = 3
merge_cdr_bp: int | None = 10000
quantile_valley: float = 0.1
prominence_perc_valley: float = 0.2

In [13]:
df

chrom,st,end,avg,idx
str,i64,i64,f64,i64
"""haplotype1-000…",615000,620000,40.32,123
"""haplotype1-000…",620000,625000,37.49,124
"""haplotype1-000…",625000,630000,36.82,125
"""haplotype1-000…",630000,635000,36.85,126
"""haplotype1-000…",635000,640000,37.68,127
…,…,…,…,…
"""haplotype2-000…",4690000,4695000,36.35,19288
"""haplotype2-000…",4695000,4700000,33.48,19289
"""haplotype2-000…",4700000,4705000,33.77,19290


In [14]:
cdr_intervals: dict[str, IntervalTree] = defaultdict(IntervalTree)
for chrom, df_chr_methyl in df.group_by(["chrom"]):
    chrom: str = chrom[0]

    plt.plot(df_chr_methyl["st"], df_chr_methyl["avg"])
    ax = plt.gca()

    df_chr_methyl_adj_groups = (
        df_chr_methyl.with_columns(brk=pl.col("end") == pl.col("st").shift(-1))
        .fill_null(True)
        .with_columns(pl.col("brk").rle_id())
        # Group contiguous intervals.
        .with_columns(
            pl.when(pl.col("brk") % 2 == 0)
            .then(pl.col("brk") + 1)
            .otherwise(pl.col("brk"))
        )
        .partition_by("brk")
    )
    valley_prom = df_chr_methyl["avg"].max() * prominence_perc_valley
    valley_quantile = df_chr_methyl["avg"].quantile(quantile_valley)

    # Per group apply optionally apply a savgol filter to denoise.
    # And find peaks within the signal.
    for df_chr_methyl_adj_grp in df_chr_methyl_adj_groups:
        df_chr_methyl_adj_grp = df_chr_methyl_adj_grp.with_row_index()

        methyl_signal = df_chr_methyl_adj_grp["avg"]
        if median_filt_window:
            methyl_signal = signal.medfilt(methyl_signal, median_filt_window)

        peaks, peak_info = signal.find_peaks(
            -methyl_signal, width=1, prominence=valley_prom
        )

        df_filtered_regions = (
            df_chr_methyl_adj_grp.filter(pl.col("index").is_in(peaks))
            .rename({"index": "index_original"})
            .with_row_index()
            # Filter peaks less than desired quantile.
            .filter(pl.col("avg") < valley_quantile)
        )
        filtered_region_idxs = set(df_filtered_regions["index"])

        for i, (l, r) in enumerate(zip(peak_info["left_ips"], peak_info["right_ips"])):
            if i not in filtered_region_idxs:
                continue

            l = df_chr_methyl_adj_grp.filter(pl.col("index") == math.floor(l)).row(
                0, named=True
            )["st"]
            r = df_chr_methyl_adj_grp.filter(pl.col("index") == math.floor(r)).row(
                0, named=True
            )["end"]

            ax.axvspan(l, r, color="red", alpha=0.5)

            if merge_cdr_bp:
                l = l - merge_cdr_bp
                r = r + merge_cdr_bp
            # Add merge distance bp.
            cdr_intervals[chrom].add(Interval(l, r))

    plt.savefig(f"{chrom}.png")
    plt.close()

In [15]:
for chrom, cdrs in cdr_intervals.items():
    print(cdrs)
    starting_intervals = len(cdrs)
    cdrs.merge_overlaps()
    print(f"Merged {starting_intervals - len(cdrs)} intervals.")
    for cdr in cdrs.iter():
        print(chrom, cdr.begin + merge_cdr_bp, cdr.end - merge_cdr_bp)

IntervalTree([Interval(2315000, 2370000), Interval(2430000, 2485000), Interval(2485000, 2575000)])
Merged 0 intervals.
haplotype1-0000004 2325000 2360000
haplotype1-0000004 2440000 2475000
haplotype1-0000004 2495000 2565000
IntervalTree([Interval(1465000, 1510000), Interval(1550000, 1590000), Interval(1665000, 1740000)])
Merged 0 intervals.
haplotype1-0000020 1475000 1500000
haplotype1-0000020 1675000 1730000
haplotype1-0000020 1560000 1580000
IntervalTree([Interval(1965000, 2080000)])
Merged 0 intervals.
haplotype2-0000055 1975000 2070000
IntervalTree([Interval(4255000, 4370000), Interval(4510000, 4550000), Interval(4560000, 4635000)])
Merged 0 intervals.
haplotype1-0000012 4520000 4540000
haplotype1-0000012 4570000 4625000
haplotype1-0000012 4265000 4360000
IntervalTree([Interval(1865000, 1925000), Interval(1950000, 2080000)])
Merged 0 intervals.
haplotype2-0000048 1960000 2070000
haplotype2-0000048 1875000 1915000
IntervalTree([Interval(935000, 985000), Interval(995000, 1060000), In